# Visualization of the different Explanations

- for each feature find images where three annotators agreed on the feature and the feature location
- extract images of the tissue and compare to find similarities

In [1]:
%load_ext autoreload
%autoreload 2

import os
import hydra
import torch
import omegaconf

import numpy as np
import seaborn as sns
import torchvision.transforms as tt

from PIL import Image
from tqdm import tqdm
from pathlib import Path
from collections import Counter, defaultdict
from sklearn.metrics import confusion_matrix
from monai.inferers import SlidingWindowInferer

from src.gleason_data import GleasonX
from src.augmentations import normalize_only_transform
from src.lightning_modul import LitSegmenter

/home/mittmann/Tools/miniconda3/envs/finalGleasonXAI/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
out_path = Path("/hdd_seahawk/00_data") / "04 GleasonXAI"

In [4]:
base_path = Path(os.environ['DATASET_LOCATION']) / "GleasonXAI"
label_level = 1
data = GleasonX(path=base_path, split='all', scaling="MicronsCalibrated", transforms=None,
                                 label_level=label_level, create_seg_masks=True, explanation_file="final_filtered_explanations_df.csv", data_split=[0.7, 0.15, 0.15], tissue_mask_kwargs={"open": False, "close": False, "flood": False})


Creating masks on the fly. Very slow!


## Collect all three rater agreeing areas

In [5]:
# -- CREATE FOLDERS --
tissue_image_path = out_path / "tissue_extraction"
tissue_image_path.mkdir(exist_ok=True)

for name in data.classes_named:
    class_dir = tissue_image_path / f"{name}"
    class_dir.mkdir(exist_ok=True)

In [6]:
name_dict = {v: k for k, v in data.classes_number_mapping.items()}

In [7]:
for i in tqdm(range(len(data))):
    img, mask, background_mask = data.get(i, False)

    np_masks = np.int8(np.array(mask))
    agreement_count = np.apply_along_axis(lambda x: np.bincount(x, minlength=10), axis=0, arr=np_masks)

    for class_number, class_agreement_array in enumerate(agreement_count):
        three_agreement_mask = (class_agreement_array == 3)#.any(axis=0)
        if not np.any(three_agreement_mask):
            continue
        else:
            area_of_interest = np.where(three_agreement_mask[..., None], img, 0)  # Keep pixels where three agree, set others to black
            output_image = Image.fromarray(area_of_interest)
            output_image.save(tissue_image_path / name_dict[class_number] / f"img_{i}.png", dpi=(1000, 1000))

  0%|          | 0/1015 [00:00<?, ?it/s]

  0%|          | 3/1015 [00:04<23:35,  1.40s/it]


KeyboardInterrupt: 

## Collect confident areas for each feature of GleasonXAI

In [7]:
model_paths = [Path(f"GleasonFinal2/label_level1/SoftDiceBalanced-{i}/version_0/checkpoints/best_model.ckpt") for i in [1, 2, 3]]

model_config = base_path / "GleasonFinal2"/"label_level1"/"SoftDiceBalanced-1"/"version_0"/"logs"/"config.yaml"
config = omegaconf.OmegaConf.load(model_config)

preds_paths = []
for path in model_paths:
    assert (base_path/path).exists(), f"Could not find {str(base_path/path)}"

TRANSFORM = normalize_only_transform
tissue_mask_kwargs =  {"open": False, "close": False, "flood": False}



In [10]:
device='cpu'
net = hydra.utils.instantiate(config.model, classes=data.num_classes)
SLIDING_WINDOW_INFERER = SlidingWindowInferer(roi_size=(512, 512), sw_batch_size=1, overlap=0.5, mode="gaussian")

In [9]:
model1 = LitSegmenter.load_from_checkpoint(str(base_path / model_paths[0]), map_location=device)
model2 = LitSegmenter.load_from_checkpoint(str(base_path / model_paths[1]), map_location=device)
model3 = LitSegmenter.load_from_checkpoint(str(base_path / model_paths[2]), map_location=device)

models = [model1, model2, model3]

/home/mittmann/Tools/miniconda3/envs/finalGleasonXAI/lib/python3.10/site-packages/pytorch_lightning/utilities/migration/utils.py:55: The loaded checkpoint was produced with Lightning v2.2.0.post0, which is newer than your current Lightning version: v2.1.3
/home/mittmann/Tools/miniconda3/envs/finalGleasonXAI/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:198: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


In [11]:
def generate_model_output(model, img, device="cpu", label_remapping=None, inferer=SLIDING_WINDOW_INFERER, transform=TRANSFORM):
    model.eval()

    img = TRANSFORM(image=img)['image']
    
    if not isinstance(img, torch.Tensor):
        img = tt.functional.to_tensor(img)
    

    if len(img.size()) == 3:
        no_batch_input = True
        img = img.unsqueeze(0)
    else:
        no_batch_input = False

    img = img.to(device)

    with torch.no_grad():
        if inferer is not None:
            out = inferer(img, model)
        else:
            out = model(img)

    # Move back and strip batch_dim
    out = out.cpu()

    if label_remapping is not None:
        out = label_remapping(out)

    if no_batch_input:
        out = out[0, ...]

    return out

In [12]:
def generate_ensemble_output(example_img):
    preds_ensemble = generate_model_output(models[0], example_img, device, None, SLIDING_WINDOW_INFERER, TRANSFORM)

    for model in models[1:]:
        out = generate_model_output(model, example_img, device, None, SLIDING_WINDOW_INFERER, TRANSFORM)
        preds_ensemble += out

    preds_ensemble = torch.nn.functional.softmax(preds_ensemble, dim=0)
    return np.array(preds_ensemble.argmax(dim=0)).astype(np.uint8)
    

In [13]:
predicted_tissue_path = out_path / "predicted_tissue"
predicted_tissue_path.mkdir(exist_ok=True)

for name in data.classes_named:
    class_dir = predicted_tissue_path / f"{name}"
    class_dir.mkdir(exist_ok=True)

In [14]:
for i in tqdm(range(len(data))):
    img, _, background_mask = data.get(i, False)

    np_seg = generate_ensemble_output(img)
    mask_np_seg = np_seg + 1
    mask_np_seg[background_mask.astype(bool)] = 0


    for class_number in range(1, data.num_classes):
        class_mask = (mask_np_seg == class_number + 1)
        class_with_bg_mask = (np_seg == class_number) 
        if not np.any(class_mask): # check if class is in foreground
            continue               # if not, continue, else extract all area (even in background)
        area_of_interest = np.where(class_with_bg_mask[..., None], img, 0)
        output_image = Image.fromarray(area_of_interest)
        output_image.save(predicted_tissue_path / name_dict[class_number] / f"img_{i}.png", dpi=(1000, 1000))

  0%|          | 0/1015 [00:00<?, ?it/s]

  0%|          | 4/1015 [00:29<2:03:04,  7.30s/it]


KeyboardInterrupt: 

## Collect overlap in annotations of classes which are often confused by the GleasonXAI

### Relevant combinations: 
annotated and predicted (see Fig. 6)
- compressed glands [2] and individual glands [1]
- glomeruloid [5] and cribriform [4]
- single cells [7] and cords [8]
- comedonecrosis [9] and cribriform [4]

NEW: annotated and predicted
- compressed glands 2 and individual glands 1
- glomeruloid 5 and poorly formed glands 3
- single cells 7 and cords 8
- comedonecrosis 9 and cribriform 4
- comedonecrosis 9 and group of tumor cells 6
- poorly formed 3 and cribriform 4
- poorly formed 3 and cords 8
- poorly formed 3 and individual glands 1

In [7]:
# -- CREATE FOLDERS --
# 2 1:
compressed_individual = out_path / "compressed_individual"
compressed_individual.mkdir(exist_ok=True)

# 5 4:
glomeruloid_cribriform = out_path / "glomeruloid_cribriform"
glomeruloid_cribriform.mkdir(exist_ok=True)

# 7 8:
single_cords = out_path / "single_cords"
single_cords.mkdir(exist_ok=True)

# 9 4:
comedo_cribri_path = out_path / "comedonecrosis_cribriform"
comedo_cribri_path.mkdir(exist_ok=True)


### Number of images with overlap / confusion between the annotators

In [14]:
# set confused features (names) and path into which the images of the overlapped labels are safed into
annotate_val = 3
predict_val = 1
image_path = comedo_cribri_path

#reset counters
predict_counter = 0
annotate_counter = 0
both_counter = 0
overlap_counter = 0

In [15]:
for i in tqdm(range(len(data))):
    img, mask, _ = data.get(i, False)

    np_masks = np.array(mask)
    # agreement_count = np.apply_along_axis(lambda x: np.bincount(x, minlength=10), axis=0, arr=np_masks)
    confusion_mask = (np_masks == predict_val).any(axis=0) & (np_masks == annotate_val).any(axis=0)

    if np.any(np_masks == predict_val):
        predict_counter += 1
    
    if np.any(np_masks == annotate_val):
        annotate_counter += 1
    
    if np.any(np_masks == predict_val) and np.any(np_masks == annotate_val):
        both_counter += 1

    if not np.any(confusion_mask):
        continue

    overlap_counter += 1
    area_of_interest = np.where(confusion_mask[..., None], img, 0)  # Keep pixels where three agree, set others to black
    # output_image = Image.fromarray(area_of_interest)
    # output_image.save(image_path / f"img_{i}.png")

print(f"{name_dict[annotate_val]}:", annotate_counter)
print(f"{name_dict[predict_val]}:", predict_counter)
print("Overlap:", overlap_counter)
print("Both:", both_counter)
        

  0%|          | 0/1015 [00:00<?, ?it/s]

100%|██████████| 1015/1015 [01:20<00:00, 12.61it/s]

poorly formed and fused glands: 696
variable sized well-formed individual and discrete glands: 547
Overlap: 307
Both: 368


#### Results:

compressed or angular discrete glands: 260  
variable sized well-formed individual and discrete glands: 547  
Overlap: 240  
Both: 256  
--> 260 images with Compressed glands  
--> in 240 (92, 31%) of those images there is overlap of the classes according to the annotators   
    (i.e. at least one pixel where one annotator said cribriform and a different one compressed glands)  

---

Glomeruloid glands: 55
poorly formed and fused glands: 696
Overlap: 44
Both: 52
--> 80%

---

Glomeruloid glands: 55  
Cribriform glands: 388  
Overlap: 26  
Both: 34  
--> 47,27%  

---

single cells: 104  
cords: 214  
Overlap: 60  
Both: 74  
--> 57,69%  

---

presence of comedonecrosis: 82  
Cribriform glands: 388  
Overlap: 21  
Both: 29  
--> 35,37%  

---

presence of comedonecrosis: 82  
solid groups of tumor cells: 228  
Overlap: 63  
Both: 66  

---

poorly formed and fused glands: 696  
Cribriform glands: 388  
Overlap: 314  
Both: 339  

---

poorly formed and fused glands: 696  
cords: 214  
Overlap: 126  
Both: 139  

---

poorly formed and fused glands: 696  
variable sized well-formed individual and discrete glands: 547  
Overlap: 307  
Both: 368  



### Number of pixels with overlap / confusion in the annotations

annotated and predicted
- compressed glands 2 and individual glands 1
- glomeruloid 5 and cribriform 4
- single cells 7 and cords 8
- comedonecrosis 9 and cribriform 4

In [17]:
predict_counter = 0
annotate_counter = 0
overlap_counter = 0

annotate_val = 3
predict_val = 0

for i in tqdm(range(len(data))):
    img, mask, _ = data.get(i, False)

    np_masks = np.array(mask)
    # agreement_count = np.apply_along_axis(lambda x: np.bincount(x, minlength=10), axis=0, arr=np_masks)
    confusion_mask = (np_masks == predict_val).any(axis=0) & (np_masks == annotate_val).any(axis=0)

    if np.any(np_masks == predict_val):
        predict_counter += np.sum((np_masks == predict_val).any(axis=0))
    
    if np.any(np_masks == annotate_val):
        annotate_counter += np.sum((np_masks == annotate_val).any(axis=0))

    if not np.any(confusion_mask):
        continue

    overlap_counter += np.sum(confusion_mask)

print(f"{name_dict[annotate_val]}:", annotate_counter)
print(f"{name_dict[predict_val]}:", predict_counter)
print("Overlap:", overlap_counter)
print("")
print(f"{overlap_counter / annotate_counter} of {name_dict[annotate_val]} overlapped by {name_dict[predict_val]}")
print(f"{overlap_counter / predict_counter} of {name_dict[predict_val]} overlapped by {name_dict[annotate_val]}")

100%|██████████| 1015/1015 [01:20<00:00, 12.56it/s]

poorly formed and fused glands: 154054357
Benign: 469102418
Overlap: 55994491

0.3634722969893023 of poorly formed and fused glands overlapped by Benign
0.11936517240463254 of Benign overlapped by poorly formed and fused glands


76.447% of compressed or angular discrete glands overlapped by variable sized well-formed individual and discrete glands  
12.808% of variable sized well-formed individual discrete glands are overlapped by compressed or angular discrete glands  

64.265% of Glomeruloid glands overlapped by poorly formed and fused glands  
04.240% of poorly formed and fused glands overlapped by Glomeruloid glands  

35.134% of Glomeruloid glands overlapped by Cribriform glands  
03.996% of Cribriform glands overlapped by Glomeruloid glands  

38.919% of single cells overlapped by cords  
09.898% of cords overlapped by single cells  

09.487% of presence of comedonecrosis overlapped by Cribriform glands  
02.421% of Cribriform glands overlapped by presence of comedonecrosis  

55.327% of presence of comedonecrosis overlapped by solid groups of tumor cells  
28.968% of solid groups of tumor cells overlapped by presence of comedonecrosis  


55.327% of presence of comedonecrosis overlapped by solid groups of tumor cells  
28.968% of solid groups of tumor cells overlapped by presence of comedonecrosis  

03.329% of Cribriform glands overlapped by solid groups of tumor cells  
06.831% of solid groups of tumor cells overlapped by Cribriform glands  

32.743% of poorly formed and fused glands overlapped by Cribriform glands  
56.456% of Cribriform glands overlapped by poorly formed and fused glands  

05.662 of poorly formed and fused glands overlapped by cords  
16.894 of cords overlapped by poorly formed and fused glands  

08.687% of poorly formed and fused glands overlapped by variable sized well-formed individual and discrete glands  
14.691% of variable sized well-formed individual and discrete glands overlapped by poorly formed and fused glands  


--> 76.447% of the pixels labeled with compressed glands were also labeled with individual glands by at least one annotator  
--> 64.265% of the pixels labeled with Glomeruloid glands were also labeled with poorly formed and fused glands by at least one annotator  
--> 35.134% of the pixels labeled with Glomeruloid glands  were also labeled with Cribriform glands by at least one annotator  
--> 38.919% of the pixels labeled with single cells  were also labeled with cords by at least one annotator  
--> 55.327% of the pixels labeled with presence of comedonecrosis were also labeled with solid groups of tumor cells by at least one annotator  

--> open question: why is comedonecrosis predicted as cribriform?

comedonecrosis to groups of tumor cells is clear (high overlap like the others)

comedo necrosis occurs in solid or cribriform glands.
--> most likely the comedonecrosis is within a label of cribriform and was therefore overwritten
--> the removal of background might remove the comedonecrosis  


### Compare Comedonecrosis to background: How much is removed

In [9]:
# 9 0:
comedo_background_path = out_path / "comedonecrosis_background"
comedo_background_path.mkdir(exist_ok=True)

In [22]:
predict_counter = 0
annotate_counter = 0
overlap_counter = 0

annotate_val = 9
predict_val = 0

for i in tqdm(range(len(data))):
    img, mask, bg_mask = data.get(i, False)

    np_masks = np.array(mask)
    # agreement_count = np.apply_along_axis(lambda x: np.bincount(x, minlength=10), axis=0, arr=np_masks)
    confusion_mask = (np_masks == annotate_val).any(axis=0) & bg_mask
    
    if np.any(np_masks == annotate_val):
        annotate_counter += np.sum((np_masks == annotate_val).any(axis=0))

    if not np.any(confusion_mask):
        continue

    overlap_counter += np.sum(confusion_mask)

print(f"{name_dict[annotate_val]}:", annotate_counter)
print(f"{name_dict[predict_val]}:", predict_counter)
print("Overlap:", overlap_counter)
print("")
print(f"{overlap_counter / annotate_counter} of {name_dict[annotate_val]} overlapped by background")
# print(f"{overlap_counter / predict_counter} of {name_dict[predict_val]} overlapped by {name_dict[annotate_val]}")

  0%|          | 0/1015 [00:00<?, ?it/s]

100%|██████████| 1015/1015 [01:18<00:00, 12.90it/s]

presence of comedonecrosis: 22802625
Benign: 0
Overlap: 1747380

0.07663065107635635 of presence of comedonecrosis overlapped by background


# Check GP 5 prediction in GP 5 annotated test set

- create test set
- for every image in test set: check for three rater annotation of any of the GP 5 classes
- if GP5 class: check if GP5 predicted
- check overlap

In [6]:
test_data = GleasonX(path=base_path, split='test', scaling="MicronsCalibrated", transforms=None,
                                 label_level=label_level, create_seg_masks=True, explanation_file="final_filtered_explanations_df.csv", data_split=[0.7, 0.15, 0.15], tissue_mask_kwargs={"open": False, "close": False, "flood": False})

Creating masks on the fly. Very slow!


In [7]:
model_paths = [Path(f"GleasonFinal2/label_level1/SoftDiceBalanced-{i}/version_0/checkpoints/best_model.ckpt") for i in [1, 2, 3]]
models = []
for model_path in model_paths:
    models.append(LitSegmenter.load_from_checkpoint(str(base_path / model_path), map_location='cpu'))

/home/mittmann/Tools/miniconda3/envs/finalGleasonXAI/lib/python3.10/site-packages/pytorch_lightning/utilities/migration/utils.py:55: The loaded checkpoint was produced with Lightning v2.2.0.post0, which is newer than your current Lightning version: v2.1.3
/home/mittmann/Tools/miniconda3/envs/finalGleasonXAI/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:198: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


In [8]:
from src.augmentations import normalize_only_transform
from src.lightning_modul import LitSegmenter
from monai.inferers import SlidingWindowInferer

In [9]:
def generate_model_output(model, img, device="cpu", label_remapping=None, transform=normalize_only_transform):
    model.eval()
    inferer = SlidingWindowInferer(roi_size=(512, 512), sw_batch_size=1, overlap=0.5, mode="gaussian")

    img = transform(image=img)['image']
    
    if not isinstance(img, torch.Tensor):
        img = tt.functional.to_tensor(img)
    
    if len(img.size()) == 3:
        no_batch_input = True
        img = img.unsqueeze(0)
    else:
        no_batch_input = False

    img = img.to(device)

    with torch.no_grad():
        if inferer is not None:
            out = inferer(img, model)
        else:
            out = model(img)

    # Move back and strip batch_dim
    out = out.cpu()

    if label_remapping is not None:
        out = label_remapping(out)

    if no_batch_input:
        out = out[0, ...]

    return out

In [18]:
def predict(img):
    device = 'cpu'
    preds_ensemble = generate_model_output(models[0], img)
    for model in models[1:]:
            out = generate_model_output(model, img, device)
            preds_ensemble += out
    preds_ensemble = torch.nn.functional.softmax(preds_ensemble, dim=0)
    np_seg = np.array(preds_ensemble.argmax(dim=0)).astype(np.uint8)
    return np_seg

relevant classes: 6, 7, 8, 9

In [34]:
num_missing_5 = 0
num_hit_5 = 0
num_no5 = 0
num_has5 = 0
num_overlap5 = 0
all_images = 0

pixels_hit = 0
pixels_total_annotated = 0
pixels_total_predicted = 0

target_classes = [6, 7, 8, 9]

In [ ]:
for i in tqdm(range(len(test_data))):
    all_images += 1
    img, mask, bg_mask = test_data.get(i, False)
    np_masks = np.array(mask)

    agreement_count = np.apply_along_axis(lambda x: np.bincount(x, minlength=10), axis=0, arr=np_masks)
    relevant_agreement_counts = agreement_count[target_classes, :]
    relevant_pixels = np.any((relevant_agreement_counts == 2) | (relevant_agreement_counts == 3), axis=0) #all pixels in the given class with 2 or 3 annotators
    
    if not np.any(relevant_pixels):
        num_no5 += 1
        continue
    num_has5 += 1

    pixels_total_annotated += np.sum(relevant_pixels)

    prediction = predict(img)
    patternfive_prediction = np.isin(prediction, [6, 7, 8, 9])
    patternfive_prediction = np.where(bg_mask, 0, patternfive_prediction) #remove pixels annotated as 5 in background

    if not np.any(patternfive_prediction):
        num_missing_5 += 1
        print("missed", i)
        continue
    num_hit_5 += 1
    pixels_total_predicted += np.sum(patternfive_prediction)

    area_of_interest = np.where(relevant_pixels, patternfive_prediction, False)  # Keep pixels annotation and prediction overlap, else false
    pixels_hit += np.sum(area_of_interest)

    if np.any(area_of_interest): 
        num_overlap5 += 1
    

  0%|          | 0/152 [00:00<?, ?it/s]

 53%|█████▎    | 81/152 [02:42<02:21,  1.99s/it]

missed 80


100%|██████████| 152/152 [04:22<00:00,  1.73s/it]


In [64]:
# check total amount of GP5 prediction on images
no_5_prediction = 0 
class_5_prediction = 0
for i in tqdm(range(len(test_data))):
    all_images += 1
    img, mask, bg_mask = test_data.get(i, False)
    np_masks = np.array(mask)

    prediction = predict(img)
    patternfive_prediction = np.isin(prediction, [6, 7, 8, 9])
    patternfive_prediction = np.where(bg_mask, 0, patternfive_prediction) #remove pixels annotated as 5 in background

    if not np.any(patternfive_prediction):
        no_5_prediction += 1
        continue
    class_5_prediction += 1

print("no:", no_5_prediction)
print("yes:", class_5_prediction)

  0%|          | 0/152 [00:00<?, ?it/s]

100%|██████████| 152/152 [12:16<00:00,  4.85s/it]

no: 82
yes: 70


In [39]:
print("-- FILES --")
print("- total files:", all_images)
print("- no 5 anno:", num_no5)
print("- has 5 anno:", num_has5)
print("- missed:", num_missing_5)
print("- hit:", num_hit_5)
print("- overlap:", num_overlap5)
print("")
print("-- PIXELS --")
print("- annotated:", pixels_total_annotated)
print("- predicted:", pixels_total_predicted)
print("- overlap:", pixels_hit)

-- FILES --
- total files: 152
- no 5 anno: 115
- has 5 anno: 37
- missed: 1
- hit: 36
- overlap: 34

-- PIXELS --
- annotated: 6813435
- predicted: 7527722
- overlap: 5732197


Missed image contained annotations for classes 6 (1 annotator) and 7 (two annotators)

# Check equal agreement pixels for classes

In [7]:
conf_matrix1_sum = np.zeros((data.num_classes, data.num_classes), dtype=int)
conf_matrix2_sum = np.zeros((data.num_classes, data.num_classes), dtype=int)

for i in tqdm(range(len(data))):
    img, mask, background_mask = data.get(i, False)

    #np_masks = np.int8(np.array(mask))
    #agreement_count = np.apply_along_axis(lambda x: np.bincount(x, minlength=10), axis=0, arr=np_masks)
    stacked_arrays = np.stack([mask[0], mask[1], mask[2]], axis=-1)

    # Identify positions where all three arrays differ
    unique_values_per_position = np.apply_along_axis(lambda x: len(np.unique(x)), axis=-1, arr=stacked_arrays)
    all_differ_mask = (unique_values_per_position == 3)

    # Extract values at the differing positions
    differing_values = stacked_arrays[all_differ_mask]

    # Create confusion matrices for each pair
    labels_true = differing_values[:, 0]
    labels_pred1 = differing_values[:, 1]
    labels_pred2 = differing_values[:, 2]

    labels_true = differing_values[:, 0]
    labels_pred1 = differing_values[:, 1]
    labels_pred2 = differing_values[:, 2]

    # Confusion matrices for each pair
    conf_matrix1_sum += confusion_matrix(labels_true, labels_pred1, labels=range(10))
    conf_matrix2_sum += confusion_matrix(labels_true, labels_pred2, labels=range(10))


  0%|          | 0/1015 [00:00<?, ?it/s]

  1%|          | 7/1015 [00:26<1:04:16,  3.83s/it]


KeyboardInterrupt: 

In [ ]:
triple_counter = Counter()
for i in tqdm(range(len(data))):
    img, mask, background_mask = data.get(i, False)

    #np_masks = np.int8(np.array(mask))
    #agreement_count = np.apply_along_axis(lambda x: np.bincount(x, minlength=10), axis=0, arr=np_masks)
    stacked_arrays = np.stack([mask[0], mask[1], mask[2]], axis=-1)
    
    triples = stacked_arrays.reshape(-1, 3)
    triples = np.sort(triples)
    
    # Convert triples to tuples and update the counter
    triple_counter.update(map(tuple, triples))


triple_frequency = dict(triple_counter)

  0%|          | 0/1015 [00:00<?, ?it/s]

100%|██████████| 1015/1015 [09:58<00:00,  1.69it/s]


In [9]:
unique_triple_frequency = {triple: count for triple, count in triple_frequency.items() if len(set(triple)) == 3}
sorted_triple_frequency = dict(sorted(unique_triple_frequency.items(), key=lambda item: item[1], reverse=True))

print("Sorted Triples:")
for triple, count in sorted_triple_frequency.items():
    print(f"{triple}: {count}")


Sorted Triples:
(0, 3, 4): 12332157
(0, 1, 3): 4435248
(0, 1, 2): 4199887
(3, 6, 9): 3591597
(0, 6, 8): 3086826
(6, 8, 9): 2810926
(3, 4, 5): 2056922
(3, 8, 9): 1999913
(0, 6, 9): 1936629
(0, 3, 6): 1862838
(0, 7, 8): 1634772
(0, 3, 8): 1621977
(6, 7, 8): 1361321
(0, 8, 9): 1093395
(0, 6, 7): 1068881
(0, 3, 5): 1048490
(4, 6, 9): 1036751
(1, 2, 3): 1016600
(3, 7, 9): 843660
(6, 7, 9): 813253
(4, 6, 7): 760673
(3, 4, 6): 602869
(1, 3, 4): 561907
(0, 1, 4): 532898
(3, 7, 8): 457284
(3, 6, 8): 453252
(0, 2, 3): 437474
(3, 6, 7): 434621
(0, 4, 5): 432921
(3, 4, 9): 363030
(0, 3, 7): 270223
(0, 4, 7): 268633
(0, 3, 9): 231950
(0, 4, 9): 224942
(1, 2, 4): 197099
(0, 7, 9): 161783
(0, 2, 4): 159276
(4, 8, 9): 159114
(0, 4, 6): 154401
(3, 4, 8): 142388
(0, 1, 5): 122844
(1, 3, 5): 107382
(0, 4, 8): 89891
(3, 4, 7): 75886
(1, 8, 9): 66604
(7, 8, 9): 58796
(1, 4, 5): 58578
(1, 7, 8): 50582
(1, 2, 5): 49377
(1, 6, 9): 46206
(2, 3, 4): 42173
(0, 2, 5): 41946
(1, 6, 7): 36284
(2, 4, 5): 18139
(1, 6

In [10]:
print(name_dict[3])
print(name_dict[4])

poorly formed and fused glands
Cribriform glands


In [26]:
total_pixels = 0
for val in sorted_triple_frequency.values():
    total_pixels += val
print(total_pixels)

print("Sorted Triples:")
for triple, count in sorted_triple_frequency.items():
    print(f"{triple}: {count / total_pixels}")

57826136
Sorted Triples:
(0, 3, 4): 0.21326268454112168
(0, 1, 3): 0.07669971239302588
(0, 1, 2): 0.07262956321342308
(3, 6, 9): 0.06211027138316833
(0, 6, 8): 0.05338115623011712
(6, 8, 9): 0.048609957269149025
(3, 4, 5): 0.03557080141062858
(3, 8, 9): 0.03458493232195214
(0, 6, 9): 0.03349054828771544
(0, 3, 6): 0.032214464407582064
(0, 7, 8): 0.02827046925632382
(0, 3, 8): 0.02804920252669139
(6, 7, 8): 0.023541621387256448
(0, 8, 9): 0.01890831854993735
(0, 6, 7): 0.018484392593688087
(0, 3, 5): 0.018131766576967894
(4, 6, 9): 0.017928761485982738
(1, 2, 3): 0.017580285841682385
(3, 7, 9): 0.014589596648823294
(6, 7, 9): 0.014063761756448676
(4, 6, 7): 0.013154484332136597
(3, 4, 6): 0.010425545293221736
(1, 3, 4): 0.00971718048046648
(0, 1, 4): 0.009215521507437398
(3, 7, 8): 0.0079079120901317
(3, 6, 8): 0.007838185833478482
(0, 2, 3): 0.007565333433311194
(3, 6, 7): 0.00751599588117041
(0, 4, 5): 0.0074865974098632495
(3, 4, 9): 0.006277957081552189
(0, 3, 7): 0.0046730253600205

In [13]:
# check 3 and 4
filtered_triple_frequency = {
    triple: count for triple, count in sorted_triple_frequency.items() if 3 in triple and 4 in triple
}

# Display the filtered results
sum_with_3_4 = 0
print("Filtered Triple Frequencies (containing 3 and 4):")
for triple, count in filtered_triple_frequency.items():
    print(triple)
    sum_with_3_4 += count
print(sum_with_3_4 / total_pixels)

Filtered Triple Frequencies (containing 3 and 4):
(0, 3, 4)
(3, 4, 5)
(3, 4, 6)
(1, 3, 4)
(3, 4, 9)
(3, 4, 8)
(3, 4, 7)
(2, 3, 4)
0.27975813566377666


In [14]:
# check 0
filtered_triple_frequency = {
    triple: count for triple, count in sorted_triple_frequency.items() if 0 in triple 
}

# Display the filtered results
sum_with_0 = 0
print("Filtered Triple Frequencies (background):")
for triple, count in filtered_triple_frequency.items():
    sum_with_0 += count
print(sum_with_0 / total_pixels)

Filtered Triple Frequencies (background):
0.6481804179342019


In [15]:
# check with mapping to grade
def map_values(value):
    if value in [1, 2]:
        return 3
    elif value in [3, 4, 5]:
        return 4
    elif value in [6, 7, 8, 9]:
        return 5
    return value

summed_triple_frequency = defaultdict(int)

for triple, count in sorted_triple_frequency.items():
    mapped_triple = tuple(map(map_values, triple))
    summed_triple_frequency[mapped_triple] += count


In [16]:
summed_triple_frequency = dict(summed_triple_frequency)
total_pixels = 0
print("Filtered Triple Frequencies (in grades):")
for triple, count in summed_triple_frequency.items():
    total_pixels += count
    print(f"{triple}: {count}")
print(total_pixels)

Filtered Triple Frequencies (in grades):
(0, 4, 4): 13813568
(0, 3, 4): 5729686
(0, 3, 3): 4199887
(4, 5, 5): 9742372
(0, 5, 5): 8982286
(5, 5, 5): 5044296
(4, 4, 4): 2056922
(0, 4, 5): 4730075
(3, 3, 4): 1263076
(4, 4, 5): 1192199
(3, 4, 4): 794682
(3, 5, 5): 217121
(3, 4, 5): 33699
(0, 3, 5): 26267
57826136


In [17]:
for triple, count in summed_triple_frequency.items():
    print(f"{triple}: {count / total_pixels}")

(0, 4, 4): 0.23888104852795283
(0, 3, 4): 0.09908471145296653
(0, 3, 3): 0.07262956321342308
(4, 5, 5): 0.16847696688570027
(0, 5, 5): 0.15533263367277383
(5, 5, 5): 0.08723211248283994
(4, 4, 4): 0.03557080141062858
(0, 4, 5): 0.08179822009895318
(3, 3, 4): 0.021842649143978772
(4, 4, 5): 0.02061695770230956
(3, 4, 4): 0.013742609397245564
(3, 5, 5): 0.0037547208756953778
(3, 4, 5): 0.0005827641674000144
(0, 3, 5): 0.00045424096813247215


In [18]:
# contain 3 and 5:
contain_3_and_5 = 0.0037547208756953778 + 0.0005827641674000144 + 0.00045424096813247215
print(contain_3_and_5)

0.004791726011227865


In [ ]:
sum_3_and_3 = 0
sum_3_and_4 = 0
sum_3_and_5 = 0
sum_4_and_4 = 0
sum_4_and_5 = 0
sum_5_and_5 = 0

# Check each triple in the summed dictionary
for triple, count in summed_triple_frequency.items():
    if 3 in triple and 4 in triple:
        sum_3_and_4 += count
    if 3 in triple and 5 in triple:
        sum_3_and_5 += count
    if 4 in triple and 5 in triple:
        sum_4_and_5 += count
    if triple.count(3) >= 2:  
        sum_3_and_3 += count
    if triple.count(4) >= 2:  
        sum_4_and_4 += count
    if triple.count(5) >= 2:  
        sum_5_and_5 += count

print(f"3 and 3:", sum_3_and_3 / total_pixels)
print(f"4 and 4:", sum_4_and_4 / total_pixels)
print(f"5 and 5:", sum_5_and_5 / total_pixels)
print("")
print(f"3 and 4:", sum_3_and_4 / total_pixels)
print(f"3 and 5:", sum_3_and_5 / total_pixels)
print(f"4 and 5:", sum_4_and_5 / total_pixels)
print("")
#print("all 3:", summed_triple_frequency[(3, 3, 3)])
print("all 4:", summed_triple_frequency[(4, 4, 4)] / total_pixels)
print("all 5:", summed_triple_frequency[(5, 5, 5)] / total_pixels)

3 and 3: 0.09447221235740184
4 and 4: 0.3088114170381365
5 and 5: 0.41479643391700943

3 and 4: 0.13525273416159087
3 and 5: 0.004791726011227864
4 and 5: 0.27147490885436304

all 4: 0.03557080141062858
all 5: 0.08723211248283994


In [20]:
sum_all_equal = 0
sum_all_different = 0

for triple, count in summed_triple_frequency.items():
    num_grades = len(set(triple))
    if num_grades == 3:
        sum_all_different += count
    elif num_grades == 1:
        sum_all_equal += count

print("all different:", sum_all_different / total_pixels)
print("all equal:", sum_all_equal / total_pixels)

all different: 0.18191993668745218
all equal: 0.12280291389346852


In [27]:
sum_two_3 = 0
sum_two_4 = 0
sum_two_5 = 0

for triple, count in summed_triple_frequency.items():
    num_grades = len(set(triple))
    if num_grades == 2:
        if triple.count(3) >= 2:  
            sum_two_3 += count
        if triple.count(4) >= 2:  
            sum_two_4 += count
        if triple.count(5) >= 2:  
            sum_two_5 += count

remaining_pixels = total_pixels - sum_all_different - sum_all_equal
print("- of remaining pixels:")
print("3:", sum_two_3 / remaining_pixels)
print("4:", sum_two_4 / remaining_pixels)
print("5:", sum_two_5 / remaining_pixels)
print("")
print("- of total pixels:")
print("3:", sum_two_3 / total_pixels)
print("4:", sum_two_4 / total_pixels)
print("5:", sum_two_5 / total_pixels)

- of remaining pixels:
3: 0.13587705627365382
4: 0.3929952478027029
5: 0.4711276959236433

- of total pixels:
3: 0.09447221235740184
4: 0.27324061562750795
5: 0.32756432143416947


In [ ]:
# of the triples with 2 times GP5 explanations, which third class is most often:
five_with_0 = 0
five_with_3 = 0
five_with_4 = 0
total_5 = 0

for triple, count in summed_triple_frequency.items():
    num_grades = len(set(triple))
    if num_grades == 2:
        if triple.count(5) == 2: 
            total_5 += count
            if 0 in triple:
                five_with_0 += count
            elif 3 in triple:
                five_with_3 += count
            elif 4 in triple:
                five_with_4 += count

print("GP 5 with:")
print("- 0:", five_with_0 / total_5)
print("- 3:", five_with_3 / total_5)
print("- 4:", five_with_4 / total_5)

GP 5 with:
- 0: 0.4742049835973696
- 3: 0.01146254530791432
- 4: 0.514332471094716


In [ ]:
# of the triples with 2 times GP4 explanations, which third class is most often:
four_with_0 = 0
four_with_3 = 0
four_with_5 = 0
total_4 = 0

for triple, count in summed_triple_frequency.items():
    num_grades = len(set(triple))
    if num_grades == 2:
        if triple.count(4) == 2: 
            total_4 += count
            if 0 in triple:
                four_with_0 += count
            elif 3 in triple:
                four_with_3 += count
            elif 4 in triple:
                four_with_5 += count

print("GP 4 with:")
print("- 0:", four_with_0 / total_4)
print("- 3:", four_with_3 / total_4)
print("- 4:", four_with_5 / total_4)

GP 4 with:
- 0: 0.8742516114573706
- 3: 0.05029489984746636
- 4: 0.07545348869516302


In [39]:
# of the triples with 2 times GP3 explanations, which third class is most often:
three_with_0 = 0
three_with_4 = 0
three_with_5 = 0
total_3 = 0

for triple, count in summed_triple_frequency.items():
    num_grades = len(set(triple))
    if num_grades == 2:
        if triple.count(3) == 2: 
            total_3 += count
            if 0 in triple:
                three_with_0 += count
            elif 4 in triple:
                three_with_4 += count
            elif 5 in triple:
                three_with_5 += count

print("GP 3 with:")
print("- 0:", three_with_0 / total_3)
print("- 4:", three_with_4 / total_3)
print("- 5:", three_with_5 / total_3)

GP 3 with:
- 0: 0.7687928693641162
- 4: 0.23120713063588386
- 5: 0.0


## Check with background removed

In [39]:
triple_counter = Counter()
for i in tqdm(range(len(data))):
    img, mask, background_mask = data.get(i, False)
    
    cleaned_masks = []
    for current_mask in mask:
        current_mask = current_mask + 1
        current_mask = np.where(~background_mask, current_mask, 0)
        cleaned_masks.append(current_mask)

    #np_masks = np.int8(np.array(mask))
    #agreement_count = np.apply_along_axis(lambda x: np.bincount(x, minlength=10), axis=0, arr=np_masks)
    stacked_arrays = np.stack([cleaned_masks[0], cleaned_masks[1], cleaned_masks[2]], axis=-1)
    
    triples = stacked_arrays.reshape(-1, 3)
    triples -= 1
    triples = np.sort(triples)
    
    # Convert triples to tuples and update the counter
    triple_counter.update(map(tuple, triples))


triple_frequency = dict(triple_counter)

  0%|          | 0/1015 [00:00<?, ?it/s]

100%|██████████| 1015/1015 [09:51<00:00,  1.72it/s]


In [41]:
unique_triple_frequency = {triple: count for triple, count in triple_frequency.items() if len(set(triple)) == 3}
sorted_triple_frequency = dict(sorted(unique_triple_frequency.items(), key=lambda item: item[1], reverse=True))

total_pixels = 0
for val in sorted_triple_frequency.values():
    total_pixels += val
print(total_pixels)

print("Sorted Triples:")
for triple, count in sorted_triple_frequency.items():
    print(f"{triple}: {count / total_pixels}")

52218501
Sorted Triples:
(0, 3, 4): 0.20440496750375886
(0, 1, 3): 0.07415007949002596
(0, 1, 2): 0.06844484103440655
(3, 6, 9): 0.0674513234303681
(0, 6, 8): 0.05551099599737649
(6, 8, 9): 0.050929535491645
(3, 8, 9): 0.03699811298681285
(0, 3, 6): 0.03399425425865825
(0, 6, 9): 0.033154743373426214
(3, 4, 5): 0.032467190124818024
(0, 3, 8): 0.03007786454842892
(0, 7, 8): 0.029784443640004144
(6, 7, 8): 0.025658645390835712
(0, 8, 9): 0.019269683746762474
(4, 6, 9): 0.018998975095052998
(0, 6, 7): 0.01899234909098597
(0, 3, 5): 0.018010723823726767
(1, 2, 3): 0.01666547264541355
(3, 7, 9): 0.015731857182189123
(6, 7, 9): 0.014099542995307354
(4, 6, 7): 0.01158710013525666
(3, 4, 6): 0.010902879038982755
(0, 1, 4): 0.00928781927309633
(1, 3, 4): 0.009166827672820406
(3, 6, 8): 0.008199775784448504
(3, 7, 8): 0.008046362724966004
(3, 6, 7): 0.007998161417923505
(0, 4, 5): 0.007332707616405917
(0, 2, 3): 0.007029673256993723
(3, 4, 9): 0.006249241049642539
(0, 4, 7): 0.005013031683923673

In [42]:
# check 3 and 4
filtered_triple_frequency = {
    triple: count for triple, count in sorted_triple_frequency.items() if 3 in triple and 4 in triple
}

# Display the filtered results
sum_with_3_4 = 0
print("Filtered Triple Frequencies (containing 3 and 4):")
for triple, count in filtered_triple_frequency.items():
    print(triple)
    sum_with_3_4 += count
print(sum_with_3_4 / total_pixels)

Filtered Triple Frequencies (containing 3 and 4):
(0, 3, 4)
(3, 4, 5)
(3, 4, 6)
(1, 3, 4)
(3, 4, 9)
(3, 4, 8)
(3, 4, 7)
(2, 3, 4)
0.2678782947063149


In [43]:
# check 0
filtered_triple_frequency = {
    triple: count for triple, count in sorted_triple_frequency.items() if 0 in triple 
}

# Display the filtered results
sum_with_0 = 0
print("Filtered Triple Frequencies (containing 3 and 4):")
for triple, count in filtered_triple_frequency.items():
    sum_with_0 += count
print(sum_with_0 / total_pixels)

Filtered Triple Frequencies (containing 3 and 4):
0.6384842031371218


In [30]:
name_dict

{0: 'Benign',
 1: 'variable sized well-formed individual and discrete glands',
 2: 'compressed or angular discrete glands',
 3: 'poorly formed and fused glands',
 4: 'Cribriform glands',
 5: 'Glomeruloid glands',
 6: 'solid groups of tumor cells',
 7: 'single cells',
 8: 'cords',
 9: 'presence of comedonecrosis'}